In [1]:
import sys
from config import *
import argparse
import glob
import pandas as pd
import nltk
import gensim
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
import itertools
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import string
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df=pd.read_csv("/home/smriti/Desktop/NLP/MITACS/Anxiety/Data/CSV/rAnxiety11.csv")

In [3]:
df.head()

,Unnamed: 0,Title,Text,Number of Comments,Date Posted
0,0,What it feels like walking into a Cafeteria. C...,NaN,9,2011-11-15 03:35:59 EST-0500
1,1,"Hey. You know what? You're going to be okay, y...","You're all very strong, impressive people. Fig...",25,2011-11-21 05:28:25 EST-0500
2,2,Just let me live!,[deleted],3,2011-11-16 02:13:52 EST-0500
3,3,"Because fuck you, brain.",NaN,5,2011-11-27 12:53:42 EST-0500
4,4,"Probably this has been done before, but I want...",NaN,6,2011-11-09 06:09:31 EST-0500


In [4]:
#Remove unnamed columns
df=df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.head()

,Title,Text,Number of Comments,Date Posted
0,What it feels like walking into a Cafeteria. C...,NaN,9,2011-11-15 03:35:59 EST-0500
1,"Hey. You know what? You're going to be okay, y...","You're all very strong, impressive people. Fig...",25,2011-11-21 05:28:25 EST-0500
2,Just let me live!,[deleted],3,2011-11-16 02:13:52 EST-0500
3,"Because fuck you, brain.",NaN,5,2011-11-27 12:53:42 EST-0500
4,"Probably this has been done before, but I want...",NaN,6,2011-11-09 06:09:31 EST-0500


In [5]:
df.shape

(201, 4)

In [6]:
#getting rid of NaN
df=df.replace(np.nan, '', regex=True)
df.head()

,Title,Text,Number of Comments,Date Posted
0,What it feels like walking into a Cafeteria. C...,,9,2011-11-15 03:35:59 EST-0500
1,"Hey. You know what? You're going to be okay, y...","You're all very strong, impressive people. Fig...",25,2011-11-21 05:28:25 EST-0500
2,Just let me live!,[deleted],3,2011-11-16 02:13:52 EST-0500
3,"Because fuck you, brain.",,5,2011-11-27 12:53:42 EST-0500
4,"Probably this has been done before, but I want...",,6,2011-11-09 06:09:31 EST-0500


In [7]:
#getting rid of deleted values
df['Text']=df['Text'].replace('[deleted]','')
df.head()

,Title,Text,Number of Comments,Date Posted
0,What it feels like walking into a Cafeteria. C...,,9,2011-11-15 03:35:59 EST-0500
1,"Hey. You know what? You're going to be okay, y...","You're all very strong, impressive people. Fig...",25,2011-11-21 05:28:25 EST-0500
2,Just let me live!,,3,2011-11-16 02:13:52 EST-0500
3,"Because fuck you, brain.",,5,2011-11-27 12:53:42 EST-0500
4,"Probably this has been done before, but I want...",,6,2011-11-09 06:09:31 EST-0500


In [8]:
df["Post"] = df["Title"] + df["Text"]
df.head()

/usr/local/lib/python3.5/dist-packages/pandas/core/computation/check.py:19: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.6.1

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


,Title,Text,Number of Comments,Date Posted,Post
0,What it feels like walking into a Cafeteria. C...,,9,2011-11-15 03:35:59 EST-0500,What it feels like walking into a Cafeteria. C...
1,"Hey. You know what? You're going to be okay, y...","You're all very strong, impressive people. Fig...",25,2011-11-21 05:28:25 EST-0500,"Hey. You know what? You're going to be okay, y..."
2,Just let me live!,,3,2011-11-16 02:13:52 EST-0500,Just let me live!
3,"Because fuck you, brain.",,5,2011-11-27 12:53:42 EST-0500,"Because fuck you, brain."
4,"Probably this has been done before, but I want...",,6,2011-11-09 06:09:31 EST-0500,"Probably this has been done before, but I want..."


In [9]:
#Now we are concerned only with the contents of Post

## STEPS IN PREPROCESSING:
1. Remove URLs present in Titles
2. Remove Punctuation
3. Remove Stopwords
4. Tokenization
5. Lemmatization

In [10]:
def remove_html(text):
    soup=BeautifulSoup(text,'lxml')
    html_free=soup.get_text()
    return html_free

In [11]:
#Remove URLs
df['Post']=df['Post'].apply(lambda x: remove_html(str(x)))

In [12]:
#Tokenize 
tokenizer=RegexpTokenizer(r'\w+')
df['Post']=df['Post'].apply(lambda x: tokenizer.tokenize(x.lower()))

In [13]:
#Remove Stopwords 
stop=stopwords.words('english')
df['Post']=df['Post'].apply(lambda x: [item for item in x if item not in stop])

In [14]:
#Lemmatize 
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
df['Post']=df['Post'].apply(lambda x: lemmatizer.lemmatize(str(x)))

In [15]:
df.head(20)

,Title,Text,Number of Comments,Date Posted,Post
0,What it feels like walking into a Cafeteria. C...,,9,2011-11-15 03:35:59 EST-0500,"['feels', 'like', 'walking', 'cafeteria', 'cro..."
1,"Hey. You know what? You're going to be okay, y...","You're all very strong, impressive people. Fig...",25,2011-11-21 05:28:25 EST-0500,"['hey', 'know', 'going', 'okay', 'going', 'get..."
2,Just let me live!,,3,2011-11-16 02:13:52 EST-0500,"['let', 'live']"
3,"Because fuck you, brain.",,5,2011-11-27 12:53:42 EST-0500,"['fuck', 'brain']"
4,"Probably this has been done before, but I want...",,6,2011-11-09 06:09:31 EST-0500,"['probably', 'done', 'wanted', 'share', 'guys']"
5,Found this while lurking. I present to you: Xa...,,5,2011-11-19 10:04:07 EST-0500,"['found', 'lurking', 'present', 'xanax', 'vide..."
6,Every time the phone rings...,,15,2011-11-29 02:41:32 EST-0500,"['every', 'time', 'phone', 'rings']"
7,We have our own meme/advice animal!,,5,2011-11-09 03:04:36 EST-0500,"['meme', 'advice', 'animal']"
8,I seem to be utterly obsessed with practicing ...,"I am a very imaginative person, able to delibe...",10,2011-11-12 19:40:11 EST-0500,"['seem', 'utterly', 'obsessed', 'practicing', ..."
9,Found this in r/motivation and thought that it...,,2,2011-11-17 10:15:30 EST-0500,"['found', 'r', 'motivation', 'thought', 'apt',..."


In [16]:
#Now that we don't need Title or Text, we drop those columns before saving the file 
df=df.drop(['Title', 'Text'], axis = 1)
df.head()

,Number of Comments,Date Posted,Post
0,9,2011-11-15 03:35:59 EST-0500,"['feels', 'like', 'walking', 'cafeteria', 'cro..."
1,25,2011-11-21 05:28:25 EST-0500,"['hey', 'know', 'going', 'okay', 'going', 'get..."
2,3,2011-11-16 02:13:52 EST-0500,"['let', 'live']"
3,5,2011-11-27 12:53:42 EST-0500,"['fuck', 'brain']"
4,6,2011-11-09 06:09:31 EST-0500,"['probably', 'done', 'wanted', 'share', 'guys']"


In [17]:
df.to_csv("PreprocessedR2011.csv")